In [3]:
# import basic modules
import sys
import os
import time
import numpy as np
from tqdm import tqdm
import gc
import torch
import argparse
import skimage.transform

# import custom modules
code_dir = '/user_data/mmhender/imStat/code/'
sys.path.append(code_dir)
from model_src import fwrf_fit as fwrf_fit
from model_src import fwrf_predict as fwrf_predict
from model_src import texture_statistics_gabor, texture_statistics_pyramid, bdcn_features, sketch_token_features
from utils import nsd_utils, roi_utils, default_paths

from model_fitting import initialize_fitting,  merge_features, arg_parser

fpX = np.float32
device = initialize_fitting.init_cuda()


#device: 1
device#: 0
device name: GeForce RTX 3090

torch: 1.8.1+cu111
cuda:  11.1
cudnn: 8005
dtype: torch.float32


In [8]:
fitting_type='texture_pyramid'
# do_avg_pool=True
subject=1
volume_space = True
up_to_sess = 1
n_ori = 4
n_sf = 4
nonlin_fn = False
# padding_mode = 'circular';
group_all_hl_feats = True; \
sample_batch_size = 50; voxel_batch_size = 100; \
zscore_features = True; ridge = True; \
shuffle_images = False; random_images = False; random_voxel_data = False; \
do_fitting = True; do_val = True; do_varpart = True; date_str = None;
shuff_rnd_seed = 0; debug = True; \
# do_pca = False; min_pct_var = 99; max_pc_to_retain = 400; map_ind = -1; \
n_prf_sd_out = 2; mult_patch_by_prf = True; \
# downsample_factor = 1.0; do_nms = False

In [5]:
if 'pyramid' in fitting_type:
    model_name = initialize_fitting.get_pyramid_model_name(ridge, n_ori, n_sf)
    feature_types_exclude = []

    if 'plus_bdcn' in fitting_type:
        model_name2 = initialize_fitting.get_bdcn_model_name(do_pca, map_ind)
        model_name = model_name + '_plus_' + model_name2

elif 'bdcn' in fitting_type:
    model_name = initialize_fitting.get_bdcn_model_name(do_pca, map_ind)

elif 'sketch_tokens' in fitting_type:
    model_name = initialize_fitting.get_sketch_tokens_model_name(do_pca)

elif 'gabor_texture' in fitting_type:

    model_name = initialize_fitting.get_gabor_texture_model_name(ridge, n_ori, n_sf)
    feature_types_exclude = []

elif 'gabor_solo' in fitting_type:

    model_name = initialize_fitting.get_gabor_solo_model_name(ridge, n_ori, n_sf)
    feature_types_exclude = ['pixel', 'simple_feature_means', 'autocorrs', 'crosscorrs']
else:     
    raise ValueError('your string for fitting_type was not recognized')

output_dir, fn2save = initialize_fitting.get_save_path(subject, volume_space, model_name, shuffle_images, random_images, random_voxel_data, debug, date_str)

# decide what voxels to use  
voxel_mask, voxel_index, voxel_roi, voxel_ncsnr, brain_nii_shape = roi_utils.get_voxel_roi_info(subject, volume_space)

sessions = np.arange(0,up_to_sess)
zscore_betas_within_sess = True
# get all data and corresponding images, in two splits. always fixed set that gets left out
trn_stim_data, trn_voxel_data, val_stim_data, val_voxel_data, \
        image_order, image_order_trn, image_order_val = nsd_utils.get_data_splits(subject, sessions=sessions, \
                                                                     voxel_mask=voxel_mask, volume_space=volume_space, \
                                                                      zscore_betas_within_sess=zscore_betas_within_sess, \
                                                                      shuffle_images=shuffle_images, random_images=random_images, \
                                                                                         random_voxel_data=random_voxel_data)


Time Stamp: Sep-17-2021_1246_08

Will save final output file to /user_data/mmhender/imStat/model_fits/S01/texture_pyramid_ridge_4ori_4sf/Sep-17-2021_1246_08_DEBUG/


Volume space: ROI defs are located at: /lab_data/tarrlab/common/datasets/NSD/nsddata/ppdata/subj01/func1pt8mm/roi

3794 voxels of overlap between kastner and prf definitions, using prf defs
unique values in retino labels:
[-1.  0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16.
 17. 18. 19. 20. 21. 22. 23. 24. 25.]
0 voxels of overlap between face and place definitions, using place defs
unique values in categ labels:
[-1.  0. 26. 27. 28. 30. 31. 32. 33.]
1535 voxels are defined (differently) in both retinotopic areas and category areas

14913 voxels are defined across all areas, and will be used for analysis

Loading numerical label/name mappings for all ROIs:
[1, 2, 3, 4, 5, 6, 7]
['V1v', 'V1d', 'V2v', 'V2d', 'V3v', 'V3d', 'hV4']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 2

In [6]:
if 'pyramid' in fitting_type:
    # Need a multiple of 8
    process_at_size=240
    trn_stim_data = skimage.transform.resize(trn_stim_data, output_shape=(trn_stim_data.shape[0],1,process_at_size, process_at_size))
    val_stim_data = skimage.transform.resize(val_stim_data, output_shape=(val_stim_data.shape[0],1,process_at_size, process_at_size))

    
# More params for fitting
holdout_size, lambdas = initialize_fitting.get_fitting_pars(trn_voxel_data, zscore_features, ridge=ridge)
# Params for the spatial aspect of the model (possible pRFs)
aperture_rf_range = 1.1
aperture, models = initialize_fitting.get_prf_models(aperture_rf_range=aperture_rf_range)    



Possible lambda values are:
[1.0000000e+00 4.2169652e+00 1.7782795e+01 7.4989418e+01 3.1622775e+02
 1.3335215e+03 5.6234131e+03 2.3713736e+04 1.0000000e+05]
most extreme RF positions:
[-0.55 -0.55  0.04]
[0.55       0.55       0.40000001]


In [9]:
# Set up the pyramid
_fmaps_fn = texture_statistics_pyramid.steerable_pyramid_extractor(pyr_height = n_sf, n_ori = n_ori)
# Initialize the "texture" model which builds on first level feature maps
_feature_extractor = texture_statistics_pyramid.texture_feature_extractor(_fmaps_fn,sample_batch_size=sample_batch_size, feature_types_exclude=feature_types_exclude, \
                                                                          n_prf_sd_out=n_prf_sd_out, aperture=aperture, do_varpart = do_varpart, \
                              group_all_hl_feats = group_all_hl_feats, device=device)
feature_info = [_feature_extractor.feature_column_labels, _feature_extractor.feature_types_include]


[]


In [12]:
images = trn_stim_data

In [16]:
tstarray = np.zeros((10000,641,875))

In [19]:
os.makedirs('/user_data/mmhender/features/pyramid_texture/')

In [20]:
tstname = '/user_data/mmhender/features/pyramid_texture/test.npy'
np.save(tstname, tstarray)

In [15]:
_feature_extractor(images[0:2], models[0], 0)

Running steerable pyramid feature extraction...
Images array shape is:
(2, 1, 240, 240)
time elapsed = 0.55261
Computing higher order correlations...
time elapsed = 4.51019
Final size of features concatenated is [2 x 641]
Feature types included are:
['pixel_stats', 'mean_magnitudes', 'mean_realparts', 'marginal_stats_lowpass_recons', 'variance_highpass_resid', 'magnitude_feature_autocorrs', 'lowpass_recon_autocorrs', 'highpass_resid_autocorrs', 'magnitude_within_scale_crosscorrs', 'real_within_scale_crosscorrs', 'magnitude_across_scale_crosscorrs', 'real_imag_across_scale_crosscorrs', 'real_spatshift_within_scale_crosscorrs', 'real_spatshift_across_scale_crosscorrs']


(tensor([[ 2.7856e-01,  2.1680e-03, -4.0659e-01,  ..., -1.4018e-03,
          -1.1751e-03, -1.7677e-03],
         [ 3.4255e-01,  1.5607e-02,  1.9106e+00,  ..., -1.9631e-03,
          -3.1372e-04, -1.3776e-03]], device='cuda:0'),
 array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  Tru